In [ ]:
import pandas as pd
import requests
from dotenv import load_dotenv
import os
import psycopg2

#### Définition de l'accès

In [ ]:
load_dotenv()

USERNAME = os.getenv("INPI_USERNAME")
PASSWORD = os.getenv("INPI_PASSWORD")

if not USERNAME or not PASSWORD:
    raise RuntimeError("INPI_USERNAME ou INPI_PASSWORD manquant dans le .env")

LOGIN_URL = "https://registre-national-entreprises.inpi.fr/api/sso/login"

payload = {
    "username": USERNAME,
    "password": PASSWORD,
}

response = requests.post(LOGIN_URL, json=payload, timeout=10)
response.raise_for_status()

token = response.json()["token"]

HEADERS = {
    "Authorization": f"Bearer {token}"
}

#### Vérification de l'accès

In [ ]:
print("Token OK :", token[:20], "...")

#### Test

In [ ]:
siren = "788653541"
url = f"https://registre-national-entreprises.inpi.fr/api/companies/{siren}/attachments"

# Récupérer et convertir directement en dict
attachments = requests.get(url, headers=HEADERS, timeout=10).json()

# Vérification rapide
if attachments:
    print("✅ Attachments OK :", list(attachments.keys()))
else:
    print("⚠️ Aucun résultat pour ce SIREN")

In [ ]:
# Exemple : récupération des bilans saisis
bilans_saisis = attachments.get("bilansSaisis", [])

print(f"Nombre de bilans saisis disponibles : {len(bilans_saisis)}")

In [ ]:
if bilans_saisis:  # Vérifie qu'il y a au moins un bilan
    bilan_id = bilans_saisis[0]["id"]
    url_bilan = f"https://registre-national-entreprises.inpi.fr/api/bilans-saisis/{bilan_id}"
    
    bilan_json = requests.get(url_bilan, headers=HEADERS, timeout=10).json()
    
    # Affichage formaté
    print("✅ Bilan récupéré :")
    print(json.dumps(bilan_json, indent=2, ensure_ascii=False))
else:
    print("⚠️ Aucun bilan saisi disponible pour ce SIREN")

In [ ]:
# Sauvegarde du bilan dans un fichier JSON
with open(f"bilan_{bilan_id}.json", "w", encoding="utf-8") as f:
    json.dump(bilan_json, f, ensure_ascii=False, indent=2)

print(f"Bilan sauvegardé : bilan_{bilan_id}.json")

In [ ]:
bilan_saisi = attachments["bilansSaisis"][0]["bilanSaisi"]

identite = bilan_saisi["bilan"]["identite"]

print("SIREN :", identite["siren"])
print("Nom société :", identite["denomination"])
print("Adresse :", identite["adresse"])
print("Date de clôture :", identite["dateClotureExercice"])
print("Type de bilan :", identite["codeTypeBilan"])

In [ ]:
# Récupérer toutes les pages du bilan
pages = bilan_saisi["bilan"]["detail"]["pages"]

# Parcourir toutes les pages et toutes les liasses pour afficher les montants
for page in pages:
    print(f"--- Page {page['numero']} ---")
    for liasse in page["liasses"]:
        code = liasse["code"]
        m1 = liasse.get("m1")
        m2 = liasse.get("m2")
        m3 = liasse.get("m3")
        m4 = liasse.get("m4")
        print(code, m1, m2, m3, m4)

In [ ]:
total_actif = None
for page in pages:
    for liasse in page["liasses"]:
        if liasse["code"] == "AT":
            total_actif = liasse.get("m3")
            break

print("Total actif exercice N :", total_actif)

In [ ]:
pages = bilan_saisi["bilan"]["detail"]["pages"]

rows = []

# Parcourir toutes les pages et toutes les liasses
for page in pages:
    page_num = page["numero"]
    for liasse in page["liasses"]:
        rows.append({
            "page": page_num,
            "code": liasse["code"],
            "m1": liasse.get("m1"),
            "m2": liasse.get("m2"),
            "m3": liasse.get("m3"),
            "m4": liasse.get("m4")
        })

# Créer le DataFrame
df = pd.DataFrame(rows)

# Afficher un aperçu
print(df.head(20))

In [ ]:
# Remplacer les chaînes vides par 0 et convertir en int
for col in ["m1", "m2", "m3", "m4"]:
    df[col] = df[col].replace("", "0").astype(int)

print(df.head(20))

In [ ]:
print("Codes disponibles dans ce bilan :", df["code"].unique())


In [ ]:
# Total actif exercice N
total_actif = df[df["code"] == "AT"]["m3"].iloc[0]
print("Total actif exercice N :", total_actif)

# Capitaux propres exercice N
if not df[df["code"] == "CS"].empty:
    capitaux_propres = df[df["code"] == "CS"]["m3"].iloc[0]
    print("Capitaux propres exercice N :", capitaux_propres)
else:
    print("Code CS introuvable dans ce bilan")

---

#### Connection BDD

In [ ]:
# Charger .env
load_dotenv()

# Récupérer l'URL depuis les variables d'environnement
DATABASE_URL = os.getenv("NEON_DATABASE_URL")
if not DATABASE_URL:
    raise ValueError("La variable NEON_DATABASE_URL n'est pas définie dans .env")

# Connexion à la BDD
conn = psycopg2.connect(DATABASE_URL)
cur = conn.cursor()
print("✅ Connexion à Néon réussie !")

#### Création Tables BDD

In [ ]:
# Société
cur.execute("""
CREATE TABLE IF NOT EXISTS societes (
    siren VARCHAR PRIMARY KEY,
    denomination VARCHAR,
    adresse VARCHAR,
    date_cloture DATE,
    type_bilan VARCHAR,
    num_depot VARCHAR,
    code_activite VARCHAR
)
""")

# Bilan
cur.execute("""
CREATE TABLE IF NOT EXISTS lignes_bilan (
    id SERIAL PRIMARY KEY,
    siren VARCHAR REFERENCES societes(siren),
    page INT,
    code VARCHAR,
    libelle VARCHAR,
    m1 BIGINT,
    m2 BIGINT,
    m3 BIGINT,
    m4 BIGINT
)
""")

conn.commit()

#### Alimentation BDD

In [ ]:
# société
cur.execute("""
INSERT INTO societes (siren, denomination, adresse, date_cloture, type_bilan, num_depot, code_activite)
VALUES (%s, %s, %s, %s, %s, %s, %s)
ON CONFLICT (siren) DO UPDATE
SET denomination = EXCLUDED.denomination,
    adresse = EXCLUDED.adresse,
    date_cloture = EXCLUDED.date_cloture,
    type_bilan = EXCLUDED.type_bilan
""", (
    identite["siren"],
    identite["denomination"],
    identite["adresse"],
    identite["dateClotureExercice"],
    identite["codeTypeBilan"],
    identite.get("numDepot"),
    identite.get("codeActivite")
))

conn.commit()

In [ ]:
pages = bilan_saisi["bilan"]["detail"]["pages"]

# Exemple de mapping codes → libellés
code_map = {
    "AJ": "Immobilisations incorporelles",
    "A1": "Terrains",
    "AT": "Total Actif",
    "CS": "Capitaux propres",
    "CO": "Résultat net",
    "BJ": "Stocks",
    "BT": "Avances et acomptes",
    "BX": "Créances clients",
    "BZ": "Autres créances",
    "CJ": "Disponibilités",
    # ajouter le reste si nécessaire
}

for page in pages:
    page_num = page["numero"]
    for liasse in page["liasses"]:
        code = liasse["code"]
        libelle = code_map.get(code, "")
        m1 = int(liasse.get("m1") or 0)
        m2 = int(liasse.get("m2") or 0)
        m3 = int(liasse.get("m3") or 0)
        m4 = int(liasse.get("m4") or 0)

        cur.execute("""
        INSERT INTO lignes_bilan (siren, page, code, libelle, m1, m2, m3, m4)
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
        """, (
            identite["siren"],
            page_num,
            code,
            libelle,
            m1, m2, m3, m4
        ))

conn.commit()

#### Vérification insertions

In [ ]:
cur.execute("SELECT * FROM lignes_bilan WHERE siren = %s LIMIT 10", (identite["siren"],))
rows = cur.fetchall()
for row in rows:
    print(row)